In [1]:
!pip install sentencepiece transformers==4.33 datasets sacremoses evaluate sacrebleu accelerate -q


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


# Translation

Translation converts a sequence of text from one language to another. It is one of several tasks you can formulate as a sequence-to-sequence problem, a powerful framework for returning some output from an input, like translation or summarization. Translation systems are commonly used for translation between different language texts, but it can also be used for speech or some combination in between like text-to-speech or speech-to-text.

This guide will show you how to:

1. Finetune [T5](https://huggingface.co/t5-small) on the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset to translate English text to French.
2. Use your finetuned model for inference.

<Tip>
The task illustrated in this tutorial is supported by the following model architectures:

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[BART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bart), [BigBird-Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bigbird_pegasus), [Blenderbot](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot), [BlenderbotSmall](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot-small), [Encoder decoder](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/encoder-decoder), [FairSeq Machine-Translation](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/fsmt), [GPTSAN-japanese](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gptsan-japanese), [LED](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/led), [LongT5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/longt5), [M2M100](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/m2m_100), [Marian](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/marian), [mBART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mbart), [MT5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mt5), [MVP](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mvp), [NLLB](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nllb), [NLLB-MOE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nllb-moe), [Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus), [PEGASUS-X](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus_x), [PLBart](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/plbart), [ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/prophetnet), [SwitchTransformers](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/switch_transformers), [T5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/t5), [XLM-ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-prophetnet)

<!--End of the generated tip-->

</Tip>

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load OPUS Books dataset

Start by loading the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset from the 🤗 Datasets library:

In [3]:
from datasets import load_dataset

dataset = load_dataset("uvci/Koumankan_mt_dyu_fr")

Generating train split:   0%|          | 0/8065 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1471 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1393 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'translation'],
        num_rows: 8065
    })
    validation: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1471
    })
    test: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1393
    })
})

In [5]:
SRC_LANG = "dyu"
TRG_LANG = "fr"
CHARS_TO_REMOVE_REGEX = '[!"&\(\),-./:;=?+.\n\[\]]'

In [6]:
# Clean data (lowercase and punctuation)
import re

def remove_special_characters(text):
    text = re.sub(CHARS_TO_REMOVE_REGEX, " ", text.lower())
    return text.strip()

def clean_text(batch):
    # process source text
    batch["translation"][SRC_LANG] = remove_special_characters(batch["translation"][SRC_LANG])
    # process target text
    batch["translation"][TRG_LANG] = remove_special_characters(batch["translation"][TRG_LANG])
    return batch

dataset = dataset.map(clean_text)
dataset

Map:   0%|          | 0/8065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1471 [00:00<?, ? examples/s]

Map:   0%|          | 0/1393 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'translation'],
        num_rows: 8065
    })
    validation: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1471
    })
    test: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1393
    })
})

Then take a look at an example:

In [7]:
dataset["train"][0]

{'ID': 'ID_18897661270129',
 'translation': {'dyu': 'a bi ji min na', 'fr': 'il boit de l’eau'}}

`translation`: an English and French translation of the text.

## Preprocess

The next step is to load a T5 tokenizer to process the English-French language pairs:

In [8]:
from transformers import AutoTokenizer

checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, src_lang="dyu_Latn" , tgt_lang="fra_Latn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Tokenize the input (English) and target (French) separately because you can't tokenize French text with a tokenizer pretrained on an English vocabulary.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [9]:
source_lang = "dyu"
target_lang = "fr"
# prefix = "translate Dyula to French: "


def preprocess_function(examples):
    inputs = [example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1471 [00:00<?, ? examples/s]

Map:   0%|          | 0/1393 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [SacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [12]:
import evaluate

metric = evaluate.load("sacrebleu")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the SacreBLEU score:

In [13]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load T5 with [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM):

In [24]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
# # Freeze the encoder
# for param in model.model.encoder.parameters():
#     param.requires_grad = False

# for name, param in model.model.encoder.named_parameters():
#     if param.requires_grad:
#         print(f"{name} is trainable")
#     else:
#         print(f"{name} is frozen")

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the SacreBLEU metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [25]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

class BestModelCallback(EarlyStoppingCallback):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.best_score = float('-inf')
        self.best_model_path = None

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if metrics['eval_bleu'] > self.best_score:
            self.best_score = metrics['eval_bleu']
            self.best_model_path = f"{args.output_dir}/checkpoint-{state.global_step}"
        return super().on_evaluate(args, state, control, metrics, **kwargs)

    def on_train_end(self, args, state, control, **kwargs):
        if self.best_model_path:
            args.push_to_hub_model_id = f"{args.push_to_hub_model_id}-best-bleu"
            control.should_save = True
            control.should_training_stop = True
            state.best_model_checkpoint = self.best_model_path
        return control

training_args = Seq2SeqTrainingArguments(
    output_dir="nllb-en-fr_v3.0",
    evaluation_strategy="steps",
    eval_steps=500,  # Evaluate every 500 steps
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=100,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
    save_strategy="steps",
    save_steps=500,  # Save checkpoint every 500 steps
)

best_model_callback = BestModelCallback(
    early_stopping_patience=5,
    early_stopping_threshold=0.01,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[best_model_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [26]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)

total_steps = 1000
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=total_steps,
    eta_min=0,  # Minimum learning rate
)

# Define a function to update the learning rate during training
def update_lr():
  scheduler.step()

In [27]:
# Training loop
for epoch in range(0, 10):
    trainer.train()
    update_lr()
    trainer.evaluate()

Step,Training Loss,Validation Loss,Bleu,Gen Len
500,2.769900,2.336852,5.536500,12.590800
1000,2.391200,2.247035,6.561600,12.271900
1500,2.193000,2.199405,7.291800,12.249500
2000,2.027500,2.185447,9.688900,11.151600
2500,1.886500,2.170183,10.118700,11.025800
3000,1.758200,2.169933,10.667600,10.913000
3500,1.649500,2.187590,10.635800,11.042100
4000,1.552600,2.202447,10.750400,11.050300
4500,1.443300,2.218714,11.173000,10.834100
5000,1.347800,2.258301,11.182400,10.777000


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.982800,2.417294,11.953100,11.354900
1000,0.920300,2.461453,12.296500,10.938800
1500,0.851500,2.500116,12.485100,11.048900
2000,0.791000,2.554486,12.125000,11.201900
2500,0.729300,2.559799,11.869900,11.109400
3000,0.683000,2.602786,12.065300,11.056400
3500,0.621000,2.653757,12.373500,11.241300
4000,0.581900,2.693492,12.063500,11.102000


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.788000,2.537440,12.146200,11.128500
1000,0.734200,2.567884,12.396000,11.045500
1500,0.675000,2.596845,12.215600,10.990500
2000,0.625500,2.646780,12.565100,10.995200
2500,0.570300,2.679420,12.175600,11.021100
3000,0.527400,2.734358,12.578500,11.169300
3500,0.485300,2.753398,12.331600,11.125100
4000,0.444900,2.794760,12.194200,10.929300
4500,0.411100,2.833006,12.245100,10.968700
5000,0.372900,2.861239,12.358000,11.211400


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.490000,2.757056,12.009400,11.199900
1000,0.447000,2.793879,12.211000,11.053700
1500,0.411100,2.813129,12.442100,11.189700
2000,0.371300,2.856413,11.828900,11.106700
2500,0.340000,2.878772,12.288400,11.070000
3000,0.306500,2.943583,12.102800,11.348100
3500,0.291200,2.949262,12.419500,11.167900
4000,0.259100,2.994941,12.183900,11.082900


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.373600,2.885897,12.227500,11.201900
1000,0.340600,2.888389,12.135400,11.178800
1500,0.313300,2.926798,12.356200,11.191000
2000,0.284500,2.965553,12.516900,10.994600
2500,0.259300,2.972893,12.445500,11.119000
3000,0.234300,3.024700,12.092600,11.062500
3500,0.211000,3.044395,11.881700,11.214100
4000,0.194300,3.088335,12.203200,11.275300
4500,0.179000,3.095911,12.428300,11.129800


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.262000,2.991434,12.347200,11.181500
1000,0.235700,3.021219,12.073900,11.094500
1500,0.213900,3.060061,12.224300,11.125100
2000,0.193100,3.073891,11.726700,11.392900
2500,0.179000,3.097089,12.055800,11.056400
3000,0.160500,3.143509,12.178900,11.221600


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.235000,3.052827,11.976900,11.292300
1000,0.214800,3.052038,12.090800,11.057100
1500,0.192000,3.080964,11.412000,11.301800
2000,0.176300,3.127833,12.021400,11.077500
2500,0.160100,3.151974,12.091900,11.170000
3000,0.145200,3.164142,11.817700,11.280800
3500,0.134500,3.194954,12.079000,11.237300


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.146800,3.179209,12.246900,11.212100
1000,0.135800,3.198141,12.165600,11.128500
1500,0.120800,3.215169,12.345200,11.222300
2000,0.110600,3.236603,12.215700,11.145500
2500,0.104200,3.261365,12.193700,11.287600
3000,0.094500,3.269589,12.334400,11.140000
3500,0.085900,3.306982,11.777000,11.054400
4000,0.078900,3.330329,11.952900,11.138700


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.112100,3.233445,12.275900,11.169300
1000,0.102300,3.248928,12.200900,11.214100
1500,0.093700,3.280941,12.200500,11.209400
2000,0.087400,3.316128,12.256100,11.122400
2500,0.080700,3.330950,12.184500,11.240700
3000,0.074500,3.336632,12.337600,11.065300
3500,0.067500,3.373309,12.383400,11.218900
4000,0.063200,3.395747,12.692000,11.200500
4500,0.058400,3.392285,12.299800,11.118300
5000,0.055100,3.427745,12.376400,11.167200


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,0.061100,3.405840,12.324000,11.087700
1000,0.057000,3.449677,12.276500,11.058500
1500,0.053000,3.442086,12.130800,11.211400
2000,0.050300,3.475070,12.368300,11.177400
2500,0.046400,3.481982,12.582800,11.163800
3000,0.042100,3.503484,12.839200,11.080200
3500,0.038400,3.503821,12.727300,11.337900
4000,0.039100,3.538073,12.392000,11.230500
4500,0.035600,3.539459,12.345800,11.259700
5000,0.036000,3.588484,12.793200,10.981600


In [28]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Adeptschneider/nllb-en-fr_v3.0/commit/a8c64f878b4f9515ae21f0bb72c09f75934d0960', commit_message='End of training', commit_description='', oid='a8c64f878b4f9515ae21f0bb72c09f75934d0960', pr_url=None, pr_revision=None, pr_num=None)

<Tip>

For a more in-depth example of how to finetune a model for translation, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation-tf.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Come up with some text you'd like to translate to another language. For T5, you need to prefix your input depending on the task you're working on. For translation from English to French, you should prefix your input as shown below:

In [ ]:
text = "i tɔgɔ bi cogodɔ"

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for translation with your model, and pass your text to it:

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="my_awesome_opus_books_model")
translator(text)

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return the `input_ids` as PyTorch tensors:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_opus_books_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

Use the [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) method to create the translation. For more details about the different text generation strategies and parameters for controlling generation, check out the [Text Generation](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation) API.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_awesome_opus_books_model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

Decode the generated token ids back into text:

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)